# Agent Memory with Firestore
This notebook demonstrates how to use Firestore as a memory layer for Vertex AI agents.

In [ ]:
from google.cloud import firestore, storage, aiplatform
from vertexai.generative_models import GenerativeModel
import json, os

PROJECT_ID = os.getenv('PROJECT_ID') or input('Enter PROJECT_ID: ')
REGION = os.getenv('REGION','us-central1')
aiplatform.init(project=PROJECT_ID, location=REGION)
db = firestore.Client(project=PROJECT_ID)
model = GenerativeModel('gemini-1.5-pro')
print('Connected to Firestore and Vertex AI.')

In [ ]:
def remember(session_id: str, key: str, value: str):
    ref = db.collection('memory').document(session_id)
    ref.set({key: value}, merge=True)
    print(f'Memory updated for {session_id}')

def recall(session_id: str):
    doc = db.collection('memory').document(session_id).get()
    return doc.to_dict() if doc.exists else {}

# Example usage
remember('session_001', 'context', 'User discussed onboarding improvements.')
print('Recalled:', recall('session_001'))

In [ ]:
# Summarize memory and save to GCS
context = recall('session_001')
prompt = f"Summarize this memory context: {json.dumps(context)}"
summary = model.generate_content(prompt).text
print('Summary:', summary)

storage_client = storage.Client(project=PROJECT_ID)
bucket_name = f"{PROJECT_ID}-memory-{REGION}"
bucket = storage_client.bucket(bucket_name)
if not bucket.exists(): bucket = storage_client.create_bucket(bucket_name, location=REGION)
bucket.blob('memory_summary.json').upload_from_string(summary)
print('Memory summary saved to Cloud Storage.')